## Neural Network Introduction #2

This exercise is adapted from https://www.springboard.com/blog/beginners-guide-neural-network-in-python-scikit-learn-0-18/

We'll use SciKit Learn's built in Breast Cancer Data Set which has several features of tumors with a labeled class indicating whether the tumor was Malignant or Benign. We will try to create a neural network model that can take in these features and attempt to predict malignant or benign labels for tumors it has not seen before. Let's go ahead and start by getting the data!

In [66]:
# from sklearn.datasets import load_breast_cancer
# cancer2 = load_breast_cancer()

import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

cancer = pd.read_csv('../data/breast.csv')
#cancer2 = load_breast_cancer()
#df_cancer2 = pd.DataFrame(cancer2['data'], columns=cancer2['feature_names'])
#df_cancer2['target'] = cancer2['target']


Check out the dataframe - what are the first few rows of data?

In [67]:
#display(cancer)
cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [68]:
# find out the total instances and number of features
#cancer['data'].shape
cancer.drop(columns=['Unnamed: 32','id'], inplace=True)
cancer.dropna(inplace=True)
#Y=cancer2.pop('diagnosis')

In [69]:
# use describe to find out more about the data
cancer.describe

<bound method NDFrame.describe of     diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0           M        17.99         10.38          122.80     1001.0   
1           M        20.57         17.77          132.90     1326.0   
2           M        19.69         21.25          130.00     1203.0   
3           M        11.42         20.38           77.58      386.1   
4           M        20.29         14.34          135.10     1297.0   
..        ...          ...           ...             ...        ...   
564         M        21.56         22.39          142.00     1479.0   
565         M        20.13         28.25          131.20     1261.0   
566         M        16.60         28.08          108.30      858.1   
567         M        20.60         29.33          140.10     1265.0   
568         B         7.76         24.54           47.92      181.0   

     smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0            0.11840           0.2

**Q:** what can you say about this dataset?

Now, set up the data (x) and labels (y)


In [70]:
dataset = cancer.values

In [71]:
dataset

array([['M', 17.99, 10.38, ..., 0.2654, 0.4601, 0.1189],
       ['M', 20.57, 17.77, ..., 0.186, 0.275, 0.08902],
       ['M', 19.69, 21.25, ..., 0.243, 0.3613, 0.08758],
       ...,
       ['M', 16.6, 28.08, ..., 0.1418, 0.2218, 0.0782],
       ['M', 20.6, 29.33, ..., 0.265, 0.4087, 0.124],
       ['B', 7.76, 24.54, ..., 0.0, 0.2871, 0.07039]], dtype=object)

In [49]:
display(dataset[:, 0])
dataset[:,1:]

array(['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'M', 'M', 'B', 'B', 'B',
       'M', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M

array([[17.99, 10.38, 122.8, ..., 0.2654, 0.4601, 0.1189],
       [20.57, 17.77, 132.9, ..., 0.186, 0.275, 0.08902],
       [19.69, 21.25, 130.0, ..., 0.243, 0.3613, 0.08758],
       ...,
       [16.6, 28.08, 108.3, ..., 0.1418, 0.2218, 0.0782],
       [20.6, 29.33, 140.1, ..., 0.265, 0.4087, 0.124],
       [7.76, 24.54, 47.92, ..., 0.0, 0.2871, 0.07039]], dtype=object)

In [72]:

dataset = cancer.values
X = dataset[:,1:].astype(float)
Y = dataset[:, 0]


#### Train Test Split
 
Let's split our data into training and testing sets, this is done easily with SciKit Learn's train_test_split function from model_selection:

In [73]:
from sklearn.model_selection import train_test_split


#X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y,random_state=1
X_train, X_test, y_train, y_test = train_test_split(X, Y)

#clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
#clf.predict_proba(X_test[:1])
#clf.predict(X_test[:5, :])
#clf.score(X_test, y_test)

#### Data Preprocessing
 
The neural network may have difficulty converging before the maximum number of iterations allowed if the data is not normalized. Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. Note that you must apply the same scaling to the test set for meaningful results. There are a lot of different methods for normalization of data, we will use the built-in StandardScaler for standardization.

In [74]:
# Import the StandardScalar library
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit only to the training data
scaler.fit(X_train)


StandardScaler()

In [75]:
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


#### Training the model
 
Now it is time to train our model. SciKit Learn makes this incredibly easy, by using estimator objects. In this case we will import our estimator (the Multi-Layer Perceptron Classifier model) from the neural_network library of SciKit-Learn!

In [76]:
from sklearn.neural_network import MLPClassifier



Next we create an instance of the model, there are a lot of parameters you can choose to define and customize here, we will only define the hidden_layer_sizes. For this parameter you pass in a tuple consisting of the number of neurons you want at each layer, where the nth entry in the tuple represents the number of neurons in the nth layer of the MLP model. There are many ways to choose these numbers, but for simplicity we will choose 3 layers with the same number of neurons as there are features in our data set:

In [77]:
# create a Multilayerperceptron classifier and call it mlp
#clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30)).fit(X_train, y_train)

Now that the model has been made we can fit the training data to our model, remember that this data has already been processed and scaled:

**Q:** What do you see in the output? What does it tell you?

#### Predictions and Evaluation
 
Now that we have a model it is time to use it to get predictions! We can do this simply with the predict() method off of our fitted model:

In [78]:
predictions = mlp.predict(X_test)

Now we can use SciKit-Learn's built in metrics such as a classification report and confusion matrix to evaluate how well our model performed:

In [79]:
predictions

array(['M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B',
       'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B',
       'M', 'M', 'M', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'M', 'M', 'M', 'B', 'M',
       'M', 'M', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B',
       'B', 'M', 'M', 'B', 'M', 'M', 'M', 'B', 'B', 'M', 'B', 'B', 'M',
       'B', 'B', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'B',
       'B', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B', 'B', 'B',
       'B', 'B', 'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'B', 'M', 'B'],
      dtype='<U1')

In [80]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

[[91  1]
 [ 4 47]]


In [81]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           B       0.96      0.99      0.97        92
           M       0.98      0.92      0.95        51

    accuracy                           0.97       143
   macro avg       0.97      0.96      0.96       143
weighted avg       0.97      0.97      0.96       143



**Q:** what conclusion can you make from the confusion matrix?

#### Weights and biases

The downside however to using a Multi-Layer Preceptron model is how difficult it is to interpret the model itself. The weights and biases won't be easily interpretable in relation to which features are important to the model itself.

To extract the MLP weights and biases after training your model, you use its public attributes coefs_ and intercepts_.

In [82]:
# Print the coefficient values and interpret it
mlp.coefs_

[array([[-1.33867147e-01,  2.41214746e-02, -6.75595930e-02,
         -1.05992984e-01,  2.67967080e-01, -1.27234647e-01,
          1.69038606e-01, -1.02617617e-01,  1.34428507e-01,
          2.50831688e-01,  8.63548520e-02,  2.69240142e-02,
          6.60489851e-02,  7.85048384e-02, -2.07734937e-01,
         -2.80951430e-02,  7.82761018e-02, -2.39462243e-01,
         -1.78230553e-01, -2.03200197e-01,  2.61503484e-01,
         -2.10302139e-01, -1.75081275e-01,  2.27229646e-01,
         -1.53447055e-01, -3.31033268e-01,  2.36665366e-01,
         -1.80819352e-01,  1.54850460e-01,  7.54174764e-02],
        [ 1.54645242e-01, -3.46630532e-01,  8.58616025e-03,
         -1.49749578e-01, -3.66250602e-01,  3.43572809e-01,
         -2.11824285e-02, -1.06391384e-01,  2.62601853e-01,
          1.35589093e-01,  2.94824367e-01,  1.85817813e-01,
          1.07890539e-01,  2.75569523e-01,  2.67318678e-01,
          1.22294227e-01,  9.97674178e-02,  2.47094539e-01,
         -1.48838502e-01,  5.52529817e-

In [83]:
# Print the intercepts values and interpret it
mlp.intercepts_[0]

array([ 0.39218585, -0.1741619 ,  0.28788132,  0.12359421,  0.17432016,
        0.20141282,  0.08126261,  0.08697066,  0.11282077,  0.12877871,
       -0.20649267,  0.31781828, -0.00608418, -0.22120836, -0.11338641,
       -0.15045294, -0.19675486,  0.29489725, -0.07492592,  0.13095989,
        0.32755442,  0.23606258, -0.11826332, -0.26835629, -0.05682167,
        0.12067691, -0.0687162 ,  0.3563786 ,  0.099912  ,  0.04430057])

**Q:** What do you understand from the two values?

#### Additional optional tasks...

select a few known supervised techniques and compare their performance. Use 10 fold cross validation